En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [27]:
file_path = "farmers-protest-tweets-2021-2-4.json"

## Data

### Importación y validación de datos
Se importa el archivo json que se ubica en el file_path declarado anteriormente, adicionalmente se realiza una primera validación general.

In [28]:
import pandas as pd

# Leer el archivo json considerando cada línea como un json por separado
df=pd.read_json(file_path,lines=True)

# Revisar la estructura general del archivo
print(df.head())

                                                 url  \
0  https://twitter.com/ArjunSinghPanam/status/136...   
1  https://twitter.com/PrdeepNain/status/13645062...   
2  https://twitter.com/parmarmaninder/status/1364...   
3  https://twitter.com/anmoldhaliwal/status/13645...   
4  https://twitter.com/KotiaPreet/status/13645061...   

                       date  \
0 2021-02-24 09:23:35+00:00   
1 2021-02-24 09:23:32+00:00   
2 2021-02-24 09:23:22+00:00   
3 2021-02-24 09:23:16+00:00   
4 2021-02-24 09:23:10+00:00   

                                             content  \
0  The world progresses while the Indian police a...   
1  #FarmersProtest \n#ModiIgnoringFarmersDeaths \...   
2  ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...   
3  @ReallySwara @rohini_sgh watch full video here...   
4  #KisanEktaMorcha #FarmersProtest #NoFarmersNoF...   

                                     renderedContent                   id  \
0  The world progresses while the Indian police a...  13645062

In [30]:
# Descripción general del DataFrame
def describe_dataframe(df):
    print("Descripción general del DataFrame:")
    print(df.info())
    print("\nPrimeras filas del DataFrame:")
    print(df.head())
    print("\nDescripción estadística de los campos numéricos:")
    print(df.describe())